**Thông tin nhóm: Nhóm 04**

**20120083 - Nguyễn Trọng Hiếu**

**20120105 - Lê Hoàng Huy**


### 1. Application description
**What is your chosen application?**

*_Thực hiện và tối ưu hóa forward-pass của convolutional layer bằng CUDA._*

<li> Fashion MNIST </li>

* Input: Ảnh được lưu dưới dạng Eigen::Matrix, số lượng cột là số lượng ảnh, số lượng hàng là thông tin của một ảnh

<div style="display: flex">

  <div style="width: 30%; text-align: center;">

| image 0 | image 1 | image 2 |
|:-------:|:-------:|:-------:|
| 1<br>6<br>2<br>1<br>2<br>5<br>6<br>0<br>4 | 3<br>1<br>6<br>2<br>8<br>9<br>7<br>4<br>0 | 7<br>1<br>1<br>4<br>1<br>8<br>5<br>0<br>7 |
  </div>
  <div style="width: 30%; color = black; text-align: center;">

*Image1*

        1 1 6
        6 2 0
        2 5 4
  </div>
  <div style="width: 30%; color = black; text-align: center;">
  
*Matrix input*

        1 1 6
        6 2 0
        2 5 4
        1 2 4
        2 8 1
        5 9 8
        6 7 5
        0 4 0
        4 0 7
  </div>
</div
<br>

*<li>* convolutional layer</li>

* Input: Một batch các ảnh dưới dạng Matrix, số lượng cột là số lượng ảnh trong batch, số lượng hàng là thông tin của một ảnh được thể nối các giá trị theo cột như mô tả bên trên.

* Output: Số lượng ảnh đầu ra với số hàng là số lượng ảnh và số cột là height_out*width_out

* Use cases: khó để lấy vị dụ ảnh thông qua C/C++

**Does it need to speed up? Why?**

Do thời gian chạy trên cpu lâu và có khả năng song song hóa trên forward-pass của convolutional layer.


### 2. Sequential implementation
**Design: Describe steps to go from input to output (don’t show code)**

Dựa vào mô hình base <a href='https://github.com/iamhankai/mini-dnn-cpp'>mini-dnn-cpp</a> tiến hành 
* Xây dựng kiến trúc <a href='https://www.kaggle.com/code/blurredmachine/lenet-architecture-a-complete-guide'>LeNet-5</a>
* Chạy trên bộ dữ liệu <a href='https://github.com/zalandoresearch/fashion-mnist'>Fashion MNIST</a>
* Tiến hành chạy thử với <a href='https://colab.research.google.com/drive/1pwaG9S3LJyYteGH4P4GQD8R0wLte-BNy?authuser=1#scrollTo=uL24K0K1_m88'>cpu trên  google-colab</a>

**Evaluate:**
* Describe your experiment setup

*_Cần tiến hành tính toán lại channel in/out và kích thước cho từng in/out ở mỗi layer trong mô hình base <a href='https://github.com/iamhankai/mini-dnn-cpp'>mini-dnn-cpp</a>._* <br>

*_Tải bộ dữ liệu fashion-MNIST và điều hướng đúng đường dẫn để có thể đọc với thiết lập sẵn có của mô hình base._*

* Run the code to see results?

*_Kết quả được thể hiện <a href='https://github.com/trhieung/CSC14120/blob/main/report_run.ipynb'>report</a>_*

* Does it run correctly?

*_Kết quả chạy đúng mong muốn của thiết lập ban đầu._*

### 3. Parallel implementation
**Analyze: Which steps do you parallelize? Why these steps?**

*_Thực hiện song song trên forward-pass của convolutional layer.
Nhận xét tính khả thi trong việc tận dụng tính toán song song của GPU để tiết kiệm thời gian._*

**Design: How do you parallelize? (don’t show code)**

*_Đánh giá về khả năng song song, nhận thấy có thể song song ở những pha sau:_*
* pha nhân image_channel với kernel tương ứng, im2col(image.col(i), data_col), đầu ra là một ma trận data_col.
* pha nhân 2 ma trận: matmul(data_col, weight)
* pha cộng bias: sau khi check size của vector bias, nhận thấy không thích hợp cho việc tiến hành song song (size = 6)
* tiến hành song song hóa toàn bộ batch được đưa vào trong forward function.

*_Lựa chọn song song của nhóm: Thực hiện 2 khả năng đầu trong 4 khả năng nêu trên_*

**Evaluate:**

* Describe your experiment setup

*_Với mục tiêu tận dụng project sẵn có và tạo ra một sản phẩm có thể ứng dụng và cải tiến. Nhóm đã tiến hành chỉnh sửa một số thông tin trên mô hình base <a href='https://github.com/iamhankai/mini-dnn-cpp'>mini-dnn-cpp</a> tận dụng được nguồi tài nguyên miễn phí google-colab._*

*_Tiến hành chỉnh sửa một số thông tin của CMakeList nhằm hỗ trợ run cuda trong project hiện tại. Các hàm hỗ trợ có thể được tạo mới dưới định .cu và .cuh tương ứng trong src/kernel. Ở các hàm trong file .cc khi sử dụng chỉ cần include .cuh tương ứng với .cu (nơi implement cuda code)_*

*_Tiến hành run project với <a href='https://github.com/trhieung/CSC14120/blob/main/report_run.ipynb'>template</a>_*
* Run the code to see results

*_Kết quả được thể hiện <a href='https://github.com/trhieung/CSC14120/blob/main/report_run.ipynb'>report</a>_*

* Does it run correctly & faster? If not, do you know why?

*_Kết quả chạy đúng khi so sánh ngược lại với cpu. Đảm bảo được time chạy nhanh hơn cpu nhưng không đáng kể (nhanh hơn khoảng <a href='https://github.com/trhieung/CSC14120/blob/main/report_run.ipynb'>2.2 lần</a> tính cho toàn bộ 1 batch)_*

### 4. Parallel implementation + optimization
#### 4.1 pha nhân image_channel với kernel tương ứng
**Version 1** <a href='https://github.com/trhieung/CSC14120/blob/main/mini-cnn-cpp-gpu/src/kernel/final.cu'>click here - the comment code</a>

**Analyze: Which parts (often: which kernels) do you optimize? Why these parts?**

*_Đây là một trong những bước có thể tiến hành song song đã được xem xét và liệt kê ra được_*

**Design: How do you optimize? (don’t show code)**
*_với điểm ảnh đầu vào, xác định các điểm ảnh đầu ra, với số thread = số điểm ảnh đầu ra trên 1 channel, khi đó mỗi thread sẽ tính toán channel_out điểm ảnh._*

*_VD với <a href='https://www.kaggle.com/code/blurredmachine/lenet-architecture-a-complete-guide'>LeNet-5</a>, ở lớp convolution thứ 2 có số channel_out = 16, khi đó ta sẽ có 10x10 threads tính toán song song, mỗi thread tính toán 16 giá trị output._*

**Evaluate:**

* Describe your experiment setup

*_Tiến hành song song hóa dựa trên code được implement sẵn trên host_*
* Run the code to see results

*_Có viết sẵn hàm test <a href='https://github.com/trhieung/CSC14120/blob/main/mini-cnn-cpp/src/layer/conv.cc'>trong phần comment cuối file</a>_*
* Does it run correctly & faster? If not, do you know why?

*_Thời gian chạy gần như song song với cpu, nguyên do là ít thread song song, thời gian cho việc chuyển đổi kiểu dữ liệu, khởi tạo, copy từ hosttodevice và ngược lại chiếm phần lớn_*

**Version 2** <a href='https://github.com/trhieung/CSC14120/blob/main/mini-cnn-cpp-gpu/src/kernel/final.cu'>click here - the uncomment code</a>

**Analyze: Which parts (often: which kernels) do you optimize? Why these parts?**

*_Đây là một trong những bước có thể tiến hành song song đã được xem xét và liệt kê ra được_*
**Design: How do you optimize? (don’t show code)**

*_Tiến hành cải tiến so với version1, mỗi thread lúc này sẽ thực hiện tính toán trên một giá trị đầu ra duy nhất của output._*

*_VD với <a href='https://www.kaggle.com/code/blurredmachine/lenet-architecture-a-complete-guide'>LeNet-5</a>, ở lớp convolution thứ 2 có số channel_out = 16, khi đó ta sẽ có 16x10x10 threads tính toán song song, mỗi thread tính toán 1 giá trị output._*
**Evaluate:**

* Describe your experiment setup

*_Tiến hành song song hóa dựa trên code được implement sẵn trên host_*
* Run the code to see results

*_Có viết sẵn hàm test <a href='https://github.com/trhieung/CSC14120/blob/main/mini-cnn-cpp/src/layer/conv.cc'>trong phần comment cuối file</a>_*
* Does it run correctly & faster? If not, do you know why?

*_Nhanh hơn rõ rệt, tầm hơn 3 lần (có thể đặt các mốc thời gian start-stop để kiểm tra!_*
#### 4.2 pha nhân 2 ma trận: matmul(data_col, weight)
**Version 1** 

**Analyze: Which parts (often: which kernels) do you optimize? Why these parts?**

**Design: How do you optimize? (don’t show code)**

Tiến hành tận dụng lại code của lab 2 - HW2_P2

**Evaluate:**

* Describe your experiment setup

*_Tiến hành song song hóa dựa trên code được implement sẵn trên host_*
* Run the code to see results

*_Có viết sẵn hàm test <a href='https://github.com/trhieung/CSC14120/blob/main/mini-cnn-cpp/src/layer/conv.cc'>trong phần comment cuối file</a>_*
* Does it run correctly & faster? If not, do you know why?

*_Kết quả chạy đúng nhưng nhanh hơn không đáng kể do kích thước ma trận khá nhỏ_*

**Version 2**

**Analyze: Which parts (often: which kernels) do you optimize? Why these parts?**

**Design: How do you optimize? (don’t show code)**

Tiến hành tận dụng lại code của lab 2 - HW2_P2 sử dụng TILE_MATMUL

**Evaluate:**

* Describe your experiment setup

*_Tiến hành song song hóa dựa trên code được implement sẵn trên host_*
* Run the code to see results

*_Có viết sẵn hàm test <a href='https://github.com/trhieung/CSC14120/blob/main/mini-cnn-cpp/src/layer/conv.cc'>trong phần comment cuối file</a>_*
* Does it run correctly & faster? If not, do you know why?

*_Kết quả chạy đúng nhưng nhanh hơn không đáng kể do kích thước ma trận khá nhỏ_*

**Note**: 
* The Eigen::Matrix "image" input is configured to store columns concatenated with the columns of a matrix. 
* The kernel implementation is designed to format the matrix input for row concatenation with rows (**_correct input_**).

|Phase|CPU|GPU <br> (Perform calculations exclusively on the optimal versio) <br> (Version 2 in each case)|
|:----:|:----:|:----:|
|Matmul with kernel <br> (the measure is caculate on one batch - 128 inputs) <br> (2 convolution layers/batch)| **3.583224** <br> (ms/operator)<br><br><br>| **1.145822** <br> (ms/operator) <br> (without including time converting from Eigen::Matrix to correct input) <br> (including the time copy data from cpu to gpu) |
|Matmul with Weight matrix <br> (the measure is caculate on one batch - 128 inputs) <br> (2 convolution layers/batch) | **1.033322** <br> (ms/operator)<br><br><br>| **0.297709** <br> (ms/operator) <br> (without including time converting from Eigen::Matrix to correct input) <br> (including the time copy data from cpu to gpu)  |
|Total optimise <br> (the measure is caculate on 50 batchs - 128 inputs/batch) <br> (2 convolution layers/batch) | **540.092310** <br> (ms/batch) <br> <br><br>| **244.952008** <br> (ms/batch) <br> (including time converting from Eigen::Matrix to correct input) <br> (including the time copy data from cpu to gpu)  |

**_[Reference link](https://colab.research.google.com/drive/1pwaG9S3LJyYteGH4P4GQD8R0wLte-BNy?authuser=3#scrollTo=pPUN-zT8OXmY)_**

### 5. Reflection
* Eachmember: What difficulties have you encountered?

*_Chạy được toàn bộ project code song song trên gpu free của colab .\_._*

*_Lộn tùm lum chiều của data vì đó giờ toàn lưu data matrix theo hàng nối hàng mà thư viện thì để theo cột nối cột_*
* Eachmember: What have you learned?

*_Học được cách run song song bằng c/c++ với nguồn tài nguyên free .\_._*
* Your team: If you had more time, what would you do?

*_Có thể optimise pha nhân image_channel với kernel tương ứng dựa trên share_const memory với data truyền vào (image) vì kích thước của một image tương đối nhỏ_*

*_Tìm hiểu sâu hơn về cơ chế lưu trữ data khi chuyển đổi kích thước trong Eigen để tiến hành thực hiện pha song song hóa cho toàn bộ batch truyền vào trong forward-conv-layer_*

* Mong muốn cá nhân của nhóm

*_Trước hết em xin nói đây là một ý tưởng cho project tuyệt vời! Tuy nhiên thời gian của tụi em phần lớn ko nằm ở việc optimise mà lại nằm ở setup làm sao cho nó chạy được và test được với những gì tụi em làm. Nếu như thầy còn tiếp tục tận dụng project này cho các khóa sau thì em mong muốn thầy có thể set up được một môi trường sẵn và template sẵn (như các bài lab) để tụi em có thể tiến hành đi sâu hơn về kỹ thuật optimise của môn. Tụi em ko nghĩ là template của tụi em đủ đẹp để tái sử dụng nhưng cũng tạm coi là ổn để thầy có thể tận dụng và build ra template như những bài lab. Tụi em cảm ơn thầy vì môn học!_*